![tower bridge](london.jpg)

London, or as the Romans called it "Londonium"! Home to [over 8.5 million residents](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/populationandhouseholdestimatesenglandandwales/census2021unroundeddata#population-and-household-estimates-england-and-wales-data) who speak over [300 languages](https://web.archive.org/web/20080924084621/http://www.cilt.org.uk/faqs/langspoken.htm). While the City of London is a little over one square mile (hence its nickname "The Square Mile"), Greater London has grown to encompass 32 boroughs spanning a total area of 606 square miles! 

![underground train leaving a platform](tube.jpg)

Given the city's roads were originally designed for horse and cart, this area and population growth has required the development of an efficient public transport system! Since the year 2000, this has been through the local government body called **Transport for London**, or *TfL*, which is managed by the London Mayor's office. Their remit covers the London Underground, Overground, Docklands Light Railway (DLR), buses, trams, river services (clipper and [Emirates Airline cable car](https://en.wikipedia.org/wiki/London_cable_car)), roads, and even taxis.

The Mayor of London's office make their data available to the public [here](https://data.london.gov.uk/dataset). In this project, you will work with a slightly modified version of a dataset containing information about public transport journey volume by transport type. 

The data has been loaded into a **Snowflake** database called `TFL` with a single table called `JOURNEYS`, including the following data:

## TFL.JOURNEYS

| Column | Definition | Data type |
|--------|------------|-----------|
| `MONTH`| Month in number format, e.g., `1` equals January | `INTEGER` |
| `YEAR` | Year | `INTEGER` |
| `DAYS` | Number of days in the given month | `INTEGER` |
| `REPORT_DATE` | Date that the data was reported | `DATE` |
| `JOURNEY_TYPE` | Method of transport used | `VARCHAR` |
| `JOURNEYS_MILLIONS` | Millions of journeys, measured in decimals | `FLOAT` |

Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

**What are the most popular transport types, measured by the total number of journeys?**

In [38]:
-- most_popular_transport_types
SELECT TFL.JOURNEYS.JOURNEY_TYPE,
	SUM(TFL.JOURNEYS.JOURNEYS_MILLIONS) AS TOTAL_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS
GROUP BY TFL.JOURNEYS.JOURNEY_TYPE 
ORDER BY TOTAL_JOURNEYS_MILLIONS DESC;


,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,Bus,24905.193947
1,Underground & DLR,15020.466544
2,Overground,1666.845666
3,TfL Rail,411.313421
4,Tram,314.689875
5,Emirates Airline,14.583718


**Which five months and years were the most popular for the Emirates Airline? Return an output containing MONTH, YEAR, and JOURNEYS_MILLIONS, with the latter rounded to two decimal places and aliased as ROUNDED_JOURNEYS_MILLIONS. Exclude null values and save the result as emirates_airline_popularity.**

In [39]:
-- emirates_airline_popularity
SELECT TFL.JOURNEYS.MONTH,
	TFL.JOURNEYS.YEAR,
	ROUND(TFL.JOURNEYS.JOURNEYS_MILLIONS,2) AS ROUNDED_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS
	
WHERE TFL.JOURNEYS.JOURNEY_TYPE = 'Emirates Airline' AND	
	ROUNDED_JOURNEYS_MILLIONS IS NOT NULL
	
ORDER BY ROUNDED_JOURNEYS_MILLIONS DESC
LIMIT 5;


,MONTH,YEAR,ROUNDED_JOURNEYS_MILLIONS
0,5,2012,0.53
1,6,2012,0.38
2,4,2012,0.24
3,5,2013,0.19
4,5,2015,0.19


**Find the five years with the lowest volume of Underground & DLR journeys, saving as least_popular_years_tube. The results should contain the columns YEAR, JOURNEY_TYPE, and TOTAL_JOURNEYS_MILLIONS.**

In [40]:
-- least_popular_years_tube

SELECT TFL.JOURNEYS.YEAR,
	TFL.JOURNEYS.JOURNEY_TYPE,
	SUM(TFL.JOURNEYS.JOURNEYS_MILLIONS) AS TOTAL_JOURNEYS_MILLIONS
FROM TFL.JOURNEYS

WHERE TFL.JOURNEYS.JOURNEY_TYPE LIKE '%Underground%'
GROUP BY TFL.JOURNEYS.YEAR, TFL.JOURNEYS.JOURNEY_TYPE
ORDER BY TOTAL_JOURNEYS_MILLIONS ASC
LIMIT 5;


,YEAR,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,2020,Underground & DLR,310.179316
1,2021,Underground & DLR,748.452544
2,2022,Underground & DLR,1064.859009
3,2010,Underground & DLR,1096.145588
4,2011,Underground & DLR,1156.647654


**Table Description**

In [42]:
DESC TABLE TFL.JOURNEYS

,name,type,kind,null?,default,primary key,unique key,check,expression,comment,policy name,privacy domain
0,MONTH,"NUMBER(38,0)",COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN
1,YEAR,"NUMBER(38,0)",COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN
2,DAYS,"NUMBER(38,0)",COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN
3,REPORT_DATE,DATE,COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN
4,JOURNEY_TYPE,VARCHAR(100),COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN
5,JOURNEYS_MILLIONS,"NUMBER(38,30)",COLUMN,Y,NaN,N,N,NaN,NaN,NaN,NaN,NaN


In [50]:
SELECT COUNT(*) AS total_records,
	COUNT(YEAR), 
	COUNT(MONTH), 
	COUNT(DAYS), 
	COUNT(JOURNEY_TYPE), 
	COUNT(JOURNEYS_MILLIONS)
FROM TFL.JOURNEYS;

,TOTAL_RECORDS,COUNT(YEAR),COUNT(MONTH),COUNT(DAYS),COUNT(JOURNEY_TYPE),COUNT(JOURNEYS_MILLIONS)
0,936,936,936,936,936,841


In [53]:
SELECT DISTINCT(JOURNEY_TYPE),	
FROM TFL.JOURNEYS;

,JOURNEY_TYPE
0,Underground & DLR
1,Bus
2,Tram
3,Emirates Airline
4,Overground
5,TfL Rail
